If you've followed earlier parts of **Python Concurrency** series, it should be clear by now that Python's GIL prevents us from utilizing all the processor cores of your computer as Python interpreter runs only one thread at a time. This means, by default, true parallelism isn't possible in Python. Also, threads are contained in a process. That is, each thread is generated by some process and an interpreter spawns only one process. However, `multiprocessing` module lets us effectively side-step the GIL issue and achieve true parallelism by running separate process on multiple cores (by running separate interpreters). 

The `multiprocessing` module includes an API for dividing work up between multiple processes based on the API for `threading`. In some cases `multiprocessing` is a drop-in replacement, and can be used instead of `threading` to take advantage of multiple CPU cores to avoid computational bottlenecks associated with Python’s global interpreter lock. The `multiprocessing` module also introduces APIs which do not have analogs in the `threading` module.


Depending on the platform, `multiprocessing` supports three ways to start a process. These _start methods_ are:

 - _spawn_ - Default on Windows. Slow. The child process will only inherit those resources necessary to run the process objects `run()` method.
 - _fork_ - Default on Unix. All resources are inherited by child process. 
 - _forkserver_ 

A trivial example of a multiprocess program is:

In [1]:
%%file multipro1.py

from multiprocessing import Process

def f(name):
    print('hello', name)

if __name__ == '__main__':
    p = Process(target=f, args=('bob',))
    p.start()
    p.join()
    
    

Writing multipro1.py


In [2]:
!python multipro1.py

hello bob


To show the individual process IDs involved, here is an expanded example:

In [3]:
%%file processid1.py

from multiprocessing import Process
import os

def info(title):
    print(title)
    print('module name:', __name__)
    print('parent process:', os.getppid())
    print('process id:', os.getpid())

def f(name):
    info('function f')
    print('hello', name)

if __name__ == '__main__':
    info('main line')
    p = Process(target=f, args=('bob',))
    p.start()
    p.join()

Writing processid1.py


In [4]:
!python processid1.py

function f
module name: __mp_main__
parent process: 1356
process id: 7156
hello bob
main line
module name: __main__
parent process: 3536
process id: 1356


On command shell, output is:

```
main line
module name: __main__
parent process: 4296
process id: 4864
function f
module name: __mp_main__
parent process: 4864
process id: 7520
hello bob
```

Notice that order is different from Notebook output.

**Important Note**

Always use the `if __name__ == '__main__'` idiom in multiprocessing codes as shown above. For an explanation of why the `if __name__ == '__main__'` part is necessary, see [Programming guidelines](https://docs.python.org/3.7/library/multiprocessing.html#multiprocessing-programming).